In [1]:
import numpy as np
import math

In [34]:
x = [[0, 1], [1, 0]]
h = [[0.70710678, 0.70710678], [0.70710678, -0.70710678]]
cx = x

In [39]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    v = []
    for i in range(2**num_qubits):
        if i == 0:
            v.append(1)
        else:
            v.append(0)
    return v

def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    
    '''#Code for single qubit gates
    I = np.identity(2)
    flag = 0
    
    if target_qubits[0] == 0:
        O = np.kron(gate_unitary, I)
        flag = 1
    elif target_qubits[0] == 1:
        O = np.kron(I, gate_unitary)
        flag = 1
    
    for i in range((total_qubits - 1) - flag):
        if flag == 1:
            O = np.kron(O, I)
        elif i == (target_qubits[0] - 1):
            O = np.kron(O, gate_unitary)
        else:
            O = np.kron(I, I)'''
            
    #Code for multi-qubit gates
    I = np.identity(2)
    # Define projection operator |0><0|

    P0x0 = np.array([
    [1, 0],
    [0, 0]
    ])

    # Define projection operator |1><1|

    P1x1 = np.array([
    [0, 0],
    [0, 1]
    ])
    
    O = np.kron(np.kron(P0x0, I), I) + np.kron(np.kron(P1x1, I), gate_unitary)
    
    return O

def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    O = get_operator(int(math.log(len(my_qpu))/math.log(2)), program[0]["gate"], program[0]["target"])
    print(O)
    final_state = np.dot(initial_state, O)
    return final_state

In [40]:
# Create "quantum computer" with 2 qubits (this is actually just a vector :) )
my_qpu = get_ground_state(3)

my_qpu

[1, 0, 0, 0, 0, 0, 0, 0]

In [43]:
my_circuit = [
{ "gate": cx, "target": [0, 2] }
]

In [44]:
# Run circuit

final_state = run_program(my_qpu, my_circuit)

print(final_state)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]
[1. 0. 0. 0. 0. 0. 0. 0.]
